In [3]:
import os
import glob

import xarray as xr
import numpy as np

In [4]:
base = '/home/hanna/lagrings/results/stats/test/'

In [5]:
glob.glob(ll + '*pixel*all*.nc')

NameError: name 'll' is not defined

In [10]:
lat = 30
lon = 0

In [12]:
xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format('r'))['min'].sel(latitude = lat, longitude = lon).values

array(11.434159, dtype=float32)

In [4]:
def min_max_scaling():
    """ Forces all values to be between 0 and 1. 
    """
    n_times, n_lat, n_lon, n_vars = dummy.shape
    transformed = np.zeros(dummy.shape)
    for j, var in enumerate(VARIABLES):

        vmin = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['min'].values
        vmax = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['max'].values
        
        if var == 'tcc':
            # Something wierd with the rotation of cloud cover values
            vmin = np.flipud(vmin)
            vmax = np.flipud(vmax)

        for i in range(n_times):
            transformed[i, :, :, j] =  (dummy[i, :, :, j]  - vmin)/(vmax-vmin)
    return transformed

def normalization(order, dummy = np.random.random(( 744, 81, 161, 5))):
    """ Normalizes the distribution. It is centered around the mean with std of 1.
    
    Subtract the mean divide by the standard deviation. """
    from sclouds.helpers import VARIABLES
    n_times, n_lat, n_lon, n_vars = dummy.shape
    VARIABLES = ['t2m', 'q', 'r', 'sp']
    transformed = np.zeros((n_times, n_lat, n_lon, 4 + order ))
    
    for j, var in enumerate(VARIABLES):

        m = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['mean'].values
        s = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['std'].values

        for i in range(n_times):
            transformed[i, :, :, j] =  (raw_data[i, :, :, j]  - m)/s    
    if order > 0:
        var = 'tcc'    
        for k in range(order):
            m = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['mean'].values
            s = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['std'].values
            # Something wierd with the rotation of cloud cover values
            m = np.flipud(m)
            s = np.flipud(s)
            for i in range(n_times):
                transformed[i, :, :, k+j+1] =  (raw_data[i, :, :, k+j+1]  - m)/s  

    return transformed

## OBS OBS : the cloud cover might be upside down

In [ ]:
def reverse_min_max_scaling():
    """ Forces all values to be between 0 and 1. 
    """
    n_times, n_lat, n_lon, n_vars = dummy.shape
    transformed = np.zeros(dummy.shape)
    for j, var in enumerate(VARIABLES):

        vmin = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['min'].values
        vmax = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['max'].values
        
        if var == 'tcc':
            # Something wierd with the rotation of cloud cover values
            vmin = np.flipud(vmin)
            vmax = np.flipud(vmax)

        for i in range(n_times):
            transformed[i, :, :, j] =  (dummy[i, :, :, j] + vmin)*(vmax-vmin)
    return transformed

def reverse_normalization(order, dummy = np.random.random(( 744, 81, 161, 5))):
    """ Normalizes the distribution. It is centered around the mean with std of 1.
    
    Subtract the mean divide by the standard deviation. """
    from sclouds.helpers import VARIABLES
    n_times, n_lat, n_lon, n_vars = dummy.shape
    
    transformed = np.zeros((n_times, n_lat, n_lon, 4 + order ))
    
    for j, var in enumerate(VARIABLES):

        m = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['mean'].values
        s = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['std'].values

        for i in range(n_times):
            transformed[i, :, :, j] =  (raw_data[i, :, :, j]  + m)*s    
    if order > 0:
        var = 'tcc'    
        for k in range(order):
            m = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['mean'].values
            s = xr.open_dataset(base + 'stats_pixel_{}_all.nc'.format(var))['std'].values
            # Something wierd with the rotation of cloud cover values
            m = np.flipud(m)
            s = np.flipud(s)
            for i in range(n_times):
                transformed[i, :, :, k+j+1] =  (raw_data[i, :, :, k+j+1]  + m)*s  

    return transformed